In [13]:
import pandas as pd

In [14]:
train = pd.read_csv("train.csv")

In [15]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [17]:
train[train['Sex'].str.match("female")].count()

PassengerId    314
Survived       314
Pclass         314
Name           314
Sex            314
Age            261
SibSp          314
Parch          314
Ticket         314
Fare           314
Cabin           97
Embarked       312
dtype: int64

In [18]:
train[train['Sex'].str.match("male")].count()

PassengerId    577
Survived       577
Pclass         577
Name           577
Sex            577
Age            453
SibSp          577
Parch          577
Ticket         577
Fare           577
Cabin          107
Embarked       577
dtype: int64

In [19]:
def fill_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return int(train[train["Pclass"] == Pclass]["Age"].mean())
    else:
        return Age

In [20]:
train["Age"] = train[["Age", "Pclass"]].apply(fill_age,axis=1)


In [21]:
train.drop("Cabin",inplace=True,axis=1)


In [22]:
train.dropna(inplace=True)

In [23]:
pd.get_dummies(train["Sex"])

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
5,0,1
6,0,1
7,0,1
8,1,0
9,1,0


In [24]:
sex = pd.get_dummies(train["Sex"],drop_first=True)

In [25]:
embarked = pd.get_dummies(train["Embarked"],drop_first=True)
pclass = pd.get_dummies(train["Pclass"],drop_first=True)


In [26]:
train = pd.concat([train,pclass,sex,embarked],axis=1)


In [27]:
train.drop(["PassengerId","Pclass","Name","Sex","Ticket","Embarked"],axis=1,inplace=True)


In [28]:
X = train.drop("Survived",axis=1)
y = train["Survived"]


In [70]:
!pip install sklearn

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)


In [30]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [40]:
X_train.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 2, 3, 'male', 'Q', 'S'], dtype='object')

In [31]:
logmodel.coef_

array([[-0.02810668, -0.22113814, -0.08966511,  0.00481737, -0.49340275,
        -1.58394673, -2.3332036 , -0.02655071, -0.26469998]])

In [32]:
logmodel.intercept_

array([2.81284544])

- coef_는 w 가중치를 의미한다.


- intercept_는 bias 또는 상수항 또는 기본점수라고 생각하면 된다. 

즉 모든 데이터는 2.8점의 기본점수를 갖고 시작한다.


- 성별이 male인 순간 -2.3점이까여서 


- 바로 0.5점 근처까지가서 사망각이다.

In [38]:
predictions = logmodel.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.81      0.91      0.86       163
           1       0.83      0.65      0.73       104

   micro avg       0.81      0.81      0.81       267
   macro avg       0.82      0.78      0.79       267
weighted avg       0.81      0.81      0.81       267



In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)


array([[149,  14],
       [ 36,  68]], dtype=int64)

In [75]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()

clf.fit(X_train, y_train)                         

predictions_clf = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_clf))


              precision    recall  f1-score   support

           0       0.83      0.87      0.85       163
           1       0.78      0.71      0.74       104

   micro avg       0.81      0.81      0.81       267
   macro avg       0.80      0.79      0.80       267
weighted avg       0.81      0.81      0.81       267



In [76]:
from sklearn.ensemble import GradientBoostingClassifier
gbt = GradientBoostingClassifier()

gbt.fit(X_train, y_train)                         

predictions_gbt = gbt.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_gbt))


              precision    recall  f1-score   support

           0       0.84      0.90      0.87       163
           1       0.82      0.72      0.77       104

   micro avg       0.83      0.83      0.83       267
   macro avg       0.83      0.81      0.82       267
weighted avg       0.83      0.83      0.83       267



In [86]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train, y_train)                         

predictions_rf = rf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_rf))



              precision    recall  f1-score   support

           0       0.81      0.89      0.85       163
           1       0.80      0.67      0.73       104

   micro avg       0.81      0.81      0.81       267
   macro avg       0.80      0.78      0.79       267
weighted avg       0.80      0.81      0.80       267



/Users/swkim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
